
<img src="C:\Users\gguevara\OneDrive - Universidad EARTH\python-analysis/logo.png">

# Plantilla para el Diseño de tuberías secundarias de riego

## Jupypter Notebook by Python 3.9.4

## Preparado por:
### Prof. Gregory Guevara Laboratorio de Fertirriego Univeridad Earth, Mayo 2021


## declaración de las variables

In [26]:
Q=40       #Caudal m3/h
S=1.5      #espaciamiento entre laterales 
L=257      #Largo del lateral 
HF=4       #Pérdidas por fricción disponibles en mca 


## Cálculo hidráulico del diámetro

In [27]:


C= 150 #rugosidad  
Salidas=L/S #definimos la cantidad de salidas que tiene el lateral de riego
F=2*Salidas/(2*Salidas-1)*((1/2.852)+0.852**0.5/(6*Salidas**2)) #Factor de Cristiansen para la ecuación de HW
Q_salida=Q/Salidas


dia=[39.8,45.9,57.38,69.46,84.58,108.72,160.08,208.42,259.75,308.05,369.7] #Tuberia SDR diametro interno

LL=L # de longitud total fija
LLL=0 #longitud 2 del diametro combinado
n=-1 #punto de inicio para el loop que busca el diametro inferior
HF3=HF #valor de inicio para la combinacion de diametros 

for j in dia: #codigo de para calcular el diametro correcto
    HF1= (1.131*10**9*(Q/C)**1.852*L*j**-4.872*F)
    if j==39.8: #definimos el diametro inferior
        jj=0
    else: #loop del diametro inferior que viene detras del diametro que se calcula
        n+=1
        jj=dia[n]
    
    Area= 3.141516*(j/2000)**2
    Vel=Q/Area/3600    
    #definimos el diametro inferior
    if Vel>3: # prueba para evitar velocidades turbulentas 
        continue   
    if HF1<HF:  #Prueba para saber que es el díametro correcto
        while HF3 >= HF:  #si el diametro es el correcto hace loop para ajustar L combinación de diametros
            L-=S
            LLL=LL-L
            Salida1=L/S
            Salida2=LLL/S
            F1=2*Salida1/(2*Salida1-1)*((1/2.852)+(0.852**0.5)/(6*Salida1**2))
            F2=2*Salida2/(2*Salida2-1)*((1/2.852)+(0.852**0.5)/(6*Salida2**2))
            Area2= 3.141516*(jj/2000)**2
            Q2=Q*L/LL #proporcion del caudal que pasasá por el diametro menor
            Vel2=Q2/Area2/3600 #velocidad del diametro menor
            HF3=(1.131*10**9*(Q2/C)**1.852*L*jj**-4.872*F1)+(1.131*10**9*(Q/C)**1.852*LLL*j**-4.872*F2 )
            
        break
    
#Salidas    
HF1=round(HF1,2)
HF3=round(HF3,2)
L=round(L,2)
LL=round(LL,2) 
LLL=round(LLL,2)
Vel=round(Vel,2)
Vel2=round(Vel2,2)

print ("***********************************************************")
print ("***********************************************************")
print ("el diametro sugerido: " + str(j) + " mm")
print ("las perdidas por fricción: " + str(HF1) + " mca")
print ("con una velocidad: " + str(Vel) + " m/s")
print ("")
print ("")
print ("también puedes combinar el dimétro  "+str(j)+" mm x "+str(LLL)+" m y " +str(jj)+" mm x "+str(L)+" m")
print ("las pérdidas son de "+str(HF3)+" mca")
print ("las velocidades son =  "+str(Vel)+" m/s x "+str(j)+" mm y " +str(Vel2)+" m/s x "+str(jj)+" mm")



***********************************************************
***********************************************************
el diametro sugerido: 84.58 mm
las perdidas por fricción: 3.6 mca
con una velocidad: 1.98 m/s


también puedes combinar el dimétro  84.58 mm x 91.5 m y 69.46 mm x 165.5 m
las pérdidas son de 3.98 mca
las velocidades son =  1.98 m/s x 84.58 mm y 1.89 m/s x 69.46 mm


## Cálculo del tiempo de avance en el lateral 

In [28]:
import pandas as pd
df=pd.DataFrame()
df["salidas"]=0
df["long_acum"]=0
df["q_tramo"]=0
df["v_tramo"]=0
df["t_tramo"]=0
df["v_tramo_comb"]=0
df["t_tramo_comb"]=0

a=range(1,int(Salidas)+1)
qq=Q
ss=0
for x in a: 
    qq=qq-Q_salida # en este vamos restado el caudal total cada vez que pasamos por una salida 
    ss=ss+S #step para el número de salidas 
    df.at[x,'salidas']=j # determinación de la columna "longitud acumulada"
    df.at[x,'long_acum']=ss # determinación de la columna "longitud acumulada"
    df.at[x,'q_tramo']=qq

#calculos del tiempo de avance sin combinación de diametros  
df["v_tramo"]=df["q_tramo"]/Area/3600
df["t_tramo"]=S/df["v_tramo"]
t_avance= round(df["t_tramo"].sum()/60,2)

#calculo del tiempo de avance con combinación de diametros 
for i in df['long_acum']:
    if i<LLL:
        df['v_tramo_comb']=df['q_tramo']/Area/3600
    else:
        df['v_tramo_comb']=df['q_tramo']/Area2/3600 
df["t_tramo_comb"]=S/df["v_tramo_comb"]




t_avance_comb= round(df["t_tramo_comb"].sum()/60,2)

print ("***********************************************************")
print ("***********************************************************")
print ("el diametro sugerido: " + str(j) + " mm")
print ("las perdidas por fricción: " + str(HF1) + " mca")
print ("con una velocidad: " + str(Vel) + " m/s")
print ("el tiempo de avance en el manifold completo es: " + str(t_avance) + " minutos")
print ("")
print ("")
print ("también puedes combinar el dimétro  "+str(j)+" mm x "+str(LLL)+" m y " +str(jj)+" mm x "+str(L)+" m")
print ("las pérdidas son de "+str(HF3)+" mca")
print ("las velocidades son =  "+str(Vel)+" m/s x "+str(j)+" mm y " +str(Vel2)+" m/s x "+str(jj)+" mm")
print ("el tiempo de avance en el manifold combinado es : " + str(t_avance_comb) + " minutos")

print ("***********************************************************")
print (df)


range(1, 172)
***********************************************************
***********************************************************
el diametro sugerido: 84.58 mm
las perdidas por fricción: 3.6 mca
con una velocidad: 1.98 m/s
el tiempo de avance en el manifold completo es: 17.92 minutos


también puedes combinar el dimétro  84.58 mm x 91.5 m y 69.46 mm x 165.5 m
las pérdidas son de 3.98 mca
las velocidades son =  1.98 m/s x 84.58 mm y 1.89 m/s x 69.46 mm
el tiempo de avance en el manifold combinado es : 12.08 minutos
***********************************************************
     salidas  long_acum    q_tramo   v_tramo     t_tramo  v_tramo_comb  \
1      84.58        1.5  39.766537  1.966077    0.762940      2.915187   
2      84.58        3.0  39.533074  1.954535    0.767446      2.898072   
3      84.58        4.5  39.299611  1.942992    0.772005      2.880958   
4      84.58        6.0  39.066148  1.931450    0.776619      2.863843   
5      84.58        7.5  38.832685  1.919907 